In [ ]:
import pandas as pd
import json
from typing import List, Dict, Any, Mapping, Optional
import numpy as np

from realestateai.data.postgres.utils import query_to_dataframe

In [ ]:
df = query_to_dataframe("""SELECT * FROM listings_bronze""")

In [ ]:
df

In [ ]:
def simple_get_field(output_dict, field_name, data_dict):
    output_dict[field_name] = data_dict.get(field_name, None)

def extract_char_fields(output_dict, field_name, data_dict):
    obj = data_dict.get(field_name)
    for k, v in obj.items():
        output_dict[k] = v.get("value")
        
        
def extract_values_fields(output_dict, field_name, data_dict):
    obj = data_dict.get(field_name)
    for k, v in obj.items():
        values = v.get("values")
        for elem in values:
            if "::" in elem:
                key, value = elem.split("::", 1)
                output_dict[key] = value
            else:
                output_dict[k] = elem
                
    
        
def extract_property_raw(output_dict: Dict[str, Any], field_name: str, data_dict: Mapping[str, Any]) -> None:
    """
    Хотим:
      - property_raw: расплющить весь объект data_dict["property_raw"]
      - buildingProperties: расплющить либо data_dict["buildingProperties"],
        либо data_dict["property_raw"]["buildingProperties"] (в твоём примере оно там)

    Ключи делаем плоскими, например:
      property_raw__condition = "TO_RENOVATION"
      property_raw__area__value = 56.25
      property_raw__buildingProperties__year = 1970
      buildingProperties__year = 1970
      buildingProperties__security__ = "ANTI_BURGLARY_DOOR"
    """

    def flatten(obj: Any, prefix: str) -> None:
        # словарь
        if isinstance(obj, Mapping):
            for kk, vv in obj.items():
                if kk == "__typename":
                    # почти всегда мусор для ML
                    continue
                flatten(vv, prefix + str(kk) + "__")
            return

        # список/кортеж
        if isinstance(obj, list) or isinstance(obj, tuple):
            output_dict[prefix] = obj
            return

        # скаляр
        if prefix.endswith("__"):
            prefix_key = prefix[:-2]
        else:
            prefix_key = prefix
        output_dict[prefix_key] = obj

    if field_name == "property_raw":
        obj = data_dict.get("property_raw")
        if obj is None:
            output_dict["property_raw"] = None
            return
        flatten(obj, "property_raw__")
        return

    if field_name == "buildingProperties":
        obj = data_dict.get("buildingProperties")
        if obj is None:
            pr = data_dict.get("property_raw")
            if isinstance(pr, Mapping):
                obj = pr.get("buildingProperties")

        if obj is None:
            output_dict["buildingProperties"] = None
            return

        flatten(obj, "buildingProperties__")
        return

    # fallback (если позже добавишь что-то ещё в этот extractor)
    obj = data_dict.get(field_name)
    if obj is None:
        output_dict[field_name] = None
        return
    flatten(obj, field_name + "__")

In [ ]:
fields_processing = [
    ("char", extract_char_fields),
    ("top_info", extract_values_fields),
    ("additional_info", extract_values_fields),
    ("ad_id", simple_get_field),
    ("url", simple_get_field), 
    ("status", simple_get_field),
    ("created_at", simple_get_field),
    ("modified_at", simple_get_field),
    ("pushed_up_at", simple_get_field),
    ("title", simple_get_field),
    ("seo_title", simple_get_field),
    ("seo_description", simple_get_field),
    ("description_text", simple_get_field),
    ("market", simple_get_field),
    ("advertiser_type", simple_get_field),
    ("advert_type", simple_get_field),
    ("exclusive_offer", simple_get_field),
    ("latitude", simple_get_field),
    ("longitude", simple_get_field),
    ("street", simple_get_field),
    ("street_number", simple_get_field),
    ("district", simple_get_field),
    ("city", simple_get_field),
    ("county", simple_get_field),
    ("province", simple_get_field),
    ("postal_code", simple_get_field),
    ("location_text", simple_get_field),
    ("features", simple_get_field),
    ("agency_name", simple_get_field),
    ("price_pln", simple_get_field),
    ("area_m2", simple_get_field),
    ("price_per_m2_pln", simple_get_field),
    ("rooms", simple_get_field),
    ("building_floors", simple_get_field),
    ("year_built", simple_get_field),
    ("rent_pln", simple_get_field),
    ("floor", simple_get_field),
    ("lift", simple_get_field),
    ("property_raw", extract_property_raw),
    ("buildingProperties", extract_property_raw),
]

In [ ]:
def process_listing(data_dict: Mapping[str, Any], fields_processing: list[tuple[str, Any]]) -> Dict[str, Any]:
    out: Dict[str, Any] = {}
    for field_name, fn in fields_processing:
        fn(out, field_name, data_dict)
    return out

In [ ]:
payloads_df = pd.DataFrame(df['payload'].apply(lambda x: process_listing(x, fields_processing)).to_list())

In [ ]:
(payloads_df.isna().sum() / len(payloads_df)).sort_values(ascending=False).iloc[:50]

In [ ]:
# drop Nan columns
payloads_df = payloads_df.drop(columns=["property_raw__rent",                           
                         "property_raw__properties__type",                     
                         "postal_code",                                        
                         "property_raw__id",                                   
                         "flat_projection",                                    
                         "street_number",                                      
                         "flat_number"])

In [ ]:
payloads_df.head()

In [ ]:
payloads_df.iloc[0].to_dict()

In [ ]:
new_df = payloads_df.copy()
# technical features
new_df['ad_id'] = new_df['ad_id'].astype("Int64")
new_df['url'] = new_df['url'].astype("string")
new_df['status'] = new_df['status'].str.lower().astype("category") # actuve or inactive
new_df['created_at'] = pd.to_datetime(new_df['created_at'], errors='coerce')
new_df['modified_at'] = pd.to_datetime(new_df['modified_at'], errors='coerce')
new_df['pushed_up_at'] = pd.to_datetime(new_df['pushed_up_at'], errors='coerce')


# appartments feature
new_df['m'] = new_df['m'].astype("Float32").round(2)
new_df['price'] = pd.to_numeric(new_df['price'], errors='coerce').round(0).astype('Int32')
new_df['floor'] = pd.to_numeric(new_df['floor'], errors='coerce').round(0).astype('Int32')
new_df['rooms_num'] = new_df['rooms_num'].astype("Int32")
new_df['price_per_m'] = pd.to_numeric(new_df['price_per_m'], errors='coerce').round(2).astype('Float32')

new_df['property_type'] = new_df['property_raw__type'].str.lower().astype("category") # ALL -> flat
new_df['property_condition'] = new_df['property_raw__condition'].str.lower().astype("category") # READY_TO_USE     376 TO_COMPLETION    331 TO_RENOVATION     52
new_df['property_ownership'] = new_df['property_raw__ownership'].str.lower().astype("category") #FULL_OWNERSHIP       673 LIMITED_OWNERSHIP     32 USUFRUCT               1
new_df['property_areas'] = new_df['property_raw__properties__areas__'] # list type -> [balcony, usable_room, garage, garden, terrace] 
new_df['property_kitchen'] = new_df['property_raw__properties__kitchen'].str.lower().astype("category") # separate 255
new_df['property_equipment'] = new_df['property_raw__properties__equipment__'] # list type -> [furniture, washing_machine, dishwasher, fridge, stove, oven]


# location features
new_df['latitude'] = pd.to_numeric(new_df['latitude'], errors='coerce').round(6).astype('Float32') # no nulls for latitude 
new_df['longitude'] = pd.to_numeric(new_df['longitude'], errors='coerce').round(6).astype('Float32') # same for longitude
new_df['street'] = new_df['street'].astype("string") # ul. Stefana Banacha 
new_df['district'] = new_df['district'].astype("string") # Prądnik Biały
new_df['city'] = new_df['city'].astype("string") # Kraków
new_df['county'] = new_df['county'].astype("string") # Kraków
new_df['province'] = new_df['province'].astype("string") # małopolskie

# text features
new_df['title'] = new_df['title'].astype("string")
new_df['seo_description'] = new_df['seo_description'].astype("string")
new_df['description_text'] = new_df['description_text'].astype("string")


# building properties

new_df['windows_type'] = new_df['windows_type'].str.lower().astype("category")
new_df['building_type'] = new_df['building_type'].str.lower().astype("category")
new_df['build_year'] = new_df['build_year'].astype("Int32")
new_df['building_floors_num'] = new_df['building_floors_num'].astype("Int32")
new_df['lift'] = new_df['lift'].astype("boolean") # True     515 False    352
new_df['building_heating'] = new_df['property_raw__buildingProperties__heating'].str.lower().astype("category") # URBAN    673 ELECTRIC  28 GAS      12 OTHER    11
new_df['building_material'] = new_df['property_raw__buildingProperties__material'].str.lower().astype("category") 
# BREEZEBLOCK 121BRICK 97CONCRETE_PLATE 58OTHER 42REINFORCED_CONCRETE 20CONCRETE 20CELLULAR_CONCRETE 14SILIKAT 7
new_df['energy_certificate'] = new_df['energy_certificate'].str.lower().astype("category") # exempt    62a         21aplus     18b          2c          1


# org property features
new_df['free_from'] = pd.to_datetime(new_df['free_from'], errors='coerce')
new_df['market'] = new_df['market'].str.lower().astype("category") # secondary    510 primary      357
new_df['construction_status'] = new_df['construction_status'].str.lower().astype("category") # ready_to_use 376 to_completion 331 to_renovation 52
new_df['advertiser_type'] = new_df['advertiser_type'].str.lower().astype("category") # business    785 private      82
new_df['advert_type'] = new_df['advert_type'].str.lower().astype("category") # AGENCY 753 PRIVATE 93 DEVELOPER_UNIT 21
new_df['agency_name'] = new_df['agency_name'].astype("string") # Semaco Real Estate 44, 153 unique values

new_df['rent'] = pd.to_numeric(new_df['rent_pln'], errors='coerce').round(2).astype('Float32') # 60% available -> 600, 800, ...

# another features
new_df['security_types'] = new_df['property_raw__buildingProperties__security__'] # list type -> [ANTI_BURGLARY_DOOR, ENTRYPHONE, MONITORING]
new_df['features'] = new_df['features'] # list type -> several string elements
new_df['building_conveniences'] = new_df['property_raw__buildingProperties__conveniences__'] # list type -> [LIFT, INTERNET] -> remove probably
new_df['internet'] = new_df['building_conveniences'].apply(lambda x: 'INTERNET' in x if isinstance(x, list) else False).astype("boolean") # True  122 False 775
new_df['garage'] = new_df['extras_types-85'].isna() # bool True     251
new_df['extra_feature'] = new_df['extras_types'].str.lower().astype('category') 
# balcony 181separate_kitchen    170 144air_conditioning    130basement  83terrace 36garden 35two_storey 23
new_df['media_types'] = new_df['media_types'].str.lower().astype('category') # cable-television    188internet 177phone 129
new_df['equipment_types'] = new_df['equipment_types'].str.lower().astype('category') 
# oven               106furniture           59tv                  58stove               48dishwasher          35washing_machine     13
new_df['remote_services'] = new_df['remote_services'].astype('category') # ?? there only "1"    201 times



new_df = new_df.drop(columns=[
    "property_raw__rent",                           
    "property_raw__properties__type",                     
    "postal_code",                                        
    "property_raw__id",                                   
    "flat_projection",                                    
    "street_number",                                      
    "flat_number"])

new_df = new_df.drop(
    columns=[
    "floor_no", 'area', "building_type-15", 'seo_title', 'exclusive_offer', 'location_text', 'price_pln', 'area_m2',
    'price_per_m2_pln', 'rooms', 'building_floors', 'year_built', 'property_raw__area__unit', 'property_raw__area__value', 
    'property_raw__type', 'property_raw__costs__',  "property_raw__condition", 'property_raw__ownership', 'property_raw__properties__areas__',
    'property_raw__properties__floor', 'property_raw__properties__rooms__', 'property_raw__properties__kitchen',
    'property_raw__properties__parking__', 'property_raw__properties__equipment__', 'property_raw__properties__numberOfRooms', 'property_raw__properties__windowsOrientation__',
    'property_raw__buildingProperties__type', 'property_raw__buildingProperties__year', 'property_raw__buildingProperties__heating',
    'property_raw__buildingProperties__windows__', 'property_raw__buildingProperties__material', 'property_raw__buildingProperties__security__',
    'property_raw__buildingProperties__conveniences__', 'property_raw__buildingProperties__numberOfFloors', 'rent_pln', 'building_ownership',
    'extras_types-85', 'extras_types', 'construction_status-67', 'property_raw__rent__value', 'property_raw__rent__currency', 'building_material-69'         
])

new_df



In [ ]:
payloads_df = payloads_df[~payloads_df["price_per_m2_pln"].isna()]

In [ ]:
from __future__ import annotations

import ast
import re
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline


# ---------- parsing helpers ----------

def parse_pln(x):
    """'1 200 zł' -> 1200.0 ; '600 zl' -> 600.0 ; NaN -> NaN"""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).lower().strip()
    if s in {"", "nan", "none", "null"}:
        return np.nan
    s = s.replace("zł", "").replace("zl", "").strip()
    s = s.replace(" ", "")
    s = re.sub(r"[^0-9.]", "", s)
    return float(s) if s else np.nan


def parse_floor_no(x):
    """'floor_4'/'FLOOR_4' -> (4,0), 'ground_floor'/'GROUND_FLOOR' -> (0,1)"""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return (np.nan, 0)
    s = str(x).strip().lower()
    if s in {"ground_floor", "parter", "0", "floor_0"}:
        return (0, 1)
    m = re.search(r"(\d+)", s)
    if m:
        return (int(m.group(1)), 0)
    return (np.nan, 0)


def _looks_multilabel_value(v) -> bool:
    if isinstance(v, (list, tuple, set)):
        return True
    if isinstance(v, str):
        s = v.strip()
        if s.startswith("[") and s.endswith("]"):
            return True
        if "|" in s:
            return True
        # осторожно с ',' — в адресах/seo оно встречается часто
        # для multi-label обычно видно много запятых + короткие токены
        if "," in s and len(s) < 120:
            return True
    return False


def parse_to_list(v):
    """Robust: list -> list[str], "['a','b']" -> ['a','b'], "a|b" -> ['a','b'], "" -> []"""
    if v is None or (isinstance(v, float) and np.isnan(v)):
        return []
    if isinstance(v, (list, tuple, set)):
        return [str(x).strip() for x in v if str(x).strip()]
    if isinstance(v, str):
        s = v.strip()
        if s == "" or s.lower() in {"nan", "none", "null"}:
            return []
        if s.startswith("[") and s.endswith("]"):
            try:
                obj = ast.literal_eval(s)
                if isinstance(obj, (list, tuple, set)):
                    return [str(x).strip() for x in obj if str(x).strip()]
                return [str(obj).strip()] if str(obj).strip() else []
            except Exception:
                # fallback: strip brackets and split by comma
                inner = s[1:-1].strip()
                if not inner:
                    return []
                parts = [p.strip().strip("'").strip('"') for p in inner.split(",")]
                return [p for p in parts if p]
        if "|" in s:
            return [p.strip() for p in s.split("|") if p.strip()]
        # comma-split only if it's not a "long text"
        if "," in s and len(s) < 120:
            return [p.strip() for p in s.split(",") if p.strip()]
        return [s]
    return [str(v).strip()] if str(v).strip() else []


def detect_multilabel_columns(df: pd.DataFrame, candidate_cols: list[str], sample_n: int = 50) -> list[str]:
    multi = []
    for col in candidate_cols:
        s = df[col].dropna()
        if s.empty:
            continue
        sample = s.sample(min(sample_n, len(s)), random_state=42).tolist()
        share = np.mean([_looks_multilabel_value(v) for v in sample])
        if share >= 0.25:
            multi.append(col)
    return multi


# ---------- MultiLabel transformer ----------

class MultiLabelBinarizerFrame(BaseEstimator, TransformerMixin):
    def __init__(self, columns: list[str]):
        self.columns = columns
        self.mlbs: dict[str, MultiLabelBinarizer] = {}

    def fit(self, X: pd.DataFrame, y=None):
        for col in self.columns:
            lists = X[col].map(parse_to_list)
            mlb = MultiLabelBinarizer()
            mlb.fit(lists)
            self.mlbs[col] = mlb
        return self

    def transform(self, X: pd.DataFrame):
        mats = []
        for col in self.columns:
            lists = X[col].map(parse_to_list)
            mats.append(self.mlbs[col].transform(lists))
        if not mats:
            return np.empty((len(X), 0))
        return np.concatenate(mats, axis=1)

    def get_feature_names_out(self):
        names = []
        for col, mlb in self.mlbs.items():
            names.extend([f"{col}__{c}" for c in mlb.classes_])
        return np.array(names, dtype=object)


# ---------- main feature prep ----------

def prepare_Xy_and_preprocessor(df: pd.DataFrame, target="price_per_m2_pln"):
    df = df.copy()

    # target
    y = df[target].astype(float)

    # drop obvious leakage + text
    drop_cols = [
        target,
        "price", "price_pln", "price_per_m",
        "url", "ad_id",
        "title", "seo_title", "seo_description", "description_text",
        "status", "location_text",
        "created_at", "modified_at", "pushed_up_at",
        "Unnamed: 15", "",  # empty col name
        # noisy split columns
        "extras_types-85", "construction_status-67", "building_material-69",
    ]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors="ignore")

    # keep one of duplicates (prefer *_m2, rooms, year_built, building_floors)
    # if these exist, drop the alternatives
    dup_drop = []
    if "area_m2" in df.columns:
        dup_drop += [c for c in ["m", "area", "property_raw__area__value"] if c in df.columns]
    if "rooms" in df.columns:
        dup_drop += [c for c in ["rooms_num", "property_raw__properties__numberOfRooms"] if c in df.columns]
    if "year_built" in df.columns:
        dup_drop += [c for c in ["build_year", "buildingProperties__year", "property_raw__buildingProperties__year"] if c in df.columns]
    if "building_floors" in df.columns:
        dup_drop += [c for c in ["building_floors_num", "buildingProperties__numberOfFloors", "property_raw__buildingProperties__numberOfFloors"] if c in df.columns]
    df = df.drop(columns=list(set(dup_drop)), errors="ignore")

    # parse rent
    if "rent" in df.columns:
        df["rent_pln_num"] = df["rent"].map(parse_pln)
        df = df.drop(columns=["rent"], errors="ignore")

    # parse floor_no
    if "floor_no" in df.columns:
        tmp = df["floor_no"].map(parse_floor_no)
        df["floor_no_num"] = [t[0] for t in tmp]
        df["is_ground_floor"] = [t[1] for t in tmp]
        df = df.drop(columns=["floor_no"], errors="ignore")

    # parse free_from date
    if "free_from" in df.columns:
        dt = pd.to_datetime(df["free_from"], errors="coerce")
        df["free_from_year"] = dt.dt.year
        df["free_from_month"] = dt.dt.month
        df = df.drop(columns=["free_from"], errors="ignore")

    # building age
    if "year_built" in df.columns:
        df["building_age"] = (2026 - pd.to_numeric(df["year_built"], errors="coerce")).clip(lower=0)

    # split types
    num_cols = df.select_dtypes(include=["number"]).columns.tolist()
    bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()
    obj_cols = df.select_dtypes(include=["object"]).columns.tolist()

    # detect multilabel among object columns (НО: исключаем geo/address-like long text)
    # например street может содержать запятые, но это не multi-label
    blocklist = {"street", "county"}  # street лучше либо убрать, либо оставить как single-cat без split
    candidate_obj = [c for c in obj_cols if c not in blocklist]

    multilabel_cols = detect_multilabel_columns(df, candidate_obj)
    cat_cols = [c for c in obj_cols if c not in multilabel_cols]

    # fill numeric NaN
    df[num_cols] = df[num_cols].fillna(df[num_cols].median(numeric_only=True))

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", "passthrough", num_cols),
            ("bool", "passthrough", bool_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore", min_frequency=2, sparse_output=True), cat_cols),
            ("mlb", MultiLabelBinarizerFrame(multilabel_cols), multilabel_cols),
        ],
        remainder="drop",
        sparse_threshold=0.3,
    )

    X = df
    info = {"num": num_cols, "bool": bool_cols, "cat": cat_cols, "multilabel": multilabel_cols}
    return X, y, preprocessor, info


In [ ]:
X, y, preprocessor, info = prepare_Xy_and_preprocessor(payloads_df)

In [ ]:
new_X = preprocessor.fit_transform(X)

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(new_X, y, test_size=0.2, random_state=42)

model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=64,
    random_state=42
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("lgbm", model),
])

pipe.fit(X_train, y_train)
print("Val score R2:", pipe.score(X_val, y_val))